# Analisando o engajamento do Instagram - pt 2 - TAGS

### O que queremos responder ?
- Qual tag mais engaja nessas publicações ?
    - Agora vamos olhar APENAS as tags
    
- Ele também dá alguns direcionamentos:
    - Podem ignorar a coluna de visualizações, queremos entender Curtidas, Comentários e Iterações.
    - Tags vazias realmente não possuem Tags (tratar como vazio)

### Importando a base

In [1]:
# Importar pandas
import pandas as pd
import numpy as np
# Formatar valores
pd.options.display.float_format='{:,.2f}'.format

In [2]:
base = pd.read_excel('08. Analisando o engajamento no Instagram.xlsx')

In [3]:
base.head()

,Tipo,Data,Curtidas,Comentários,Visualizações,Tags,Pessoas,Campanhas,Carrossel,Interacoes
0,Foto,2021-09-11,2858,16,NaN,Loja,N,N,NaN,2874
1,Foto,2021-09-11,2930,28,NaN,Loja/Produtos,N,N,NaN,2958
2,Foto,2021-09-11,2807,9,NaN,Loja,N,N,NaN,2816
3,Vídeo,2021-09-12,5115,49,"82,878.00",Produtos,N,N,NaN,5164
4,Foto,2021-09-13,4392,45,NaN,Produtos,S,N,NaN,4437


In [4]:
base = base.drop("Visualizações", axis=1)

In [5]:
base.head()

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes
0,Foto,2021-09-11,2858,16,Loja,N,N,NaN,2874
1,Foto,2021-09-11,2930,28,Loja/Produtos,N,N,NaN,2958
2,Foto,2021-09-11,2807,9,Loja,N,N,NaN,2816
3,Vídeo,2021-09-12,5115,49,Produtos,N,N,NaN,5164
4,Foto,2021-09-13,4392,45,Produtos,S,N,NaN,4437


Podemos observar que existem linhas que possuem 2 tags no mesmo campo e para que seja possível analisar de melhor forma esses dados, vamos separar essas linhas que possuem 2 tags em 2 colunas
- Para isso vamos utilizar o Split para separar as tags em itens de uma lista
- Na sequencia vamos utilizar o Explode que transforma cada item dessa lista em uma linha no nosso DataFrame 

Com isso vamos conseguir "duplicar" a linha que possue mais de uma tag, sendo que a linha 1 va ter os dados com a tag X e a linha 2 os dados com a tag Y.

In [6]:
# Transformando a nossa coluna de Tags em lista de tags
base.Tags = base.Tags.str.split('/')
base.head()

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes
0,Foto,2021-09-11,2858,16,[Loja],N,N,NaN,2874
1,Foto,2021-09-11,2930,28,"[Loja, Produtos]",N,N,NaN,2958
2,Foto,2021-09-11,2807,9,[Loja],N,N,NaN,2816
3,Vídeo,2021-09-12,5115,49,[Produtos],N,N,NaN,5164
4,Foto,2021-09-13,4392,45,[Produtos],S,N,NaN,4437


In [7]:
base = base.explode("Tags")
base.head()

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes
0,Foto,2021-09-11,2858,16,Loja,N,N,NaN,2874
1,Foto,2021-09-11,2930,28,Loja,N,N,NaN,2958
1,Foto,2021-09-11,2930,28,Produtos,N,N,NaN,2958
2,Foto,2021-09-11,2807,9,Loja,N,N,NaN,2816
3,Vídeo,2021-09-12,5115,49,Produtos,N,N,NaN,5164


# SUPER IMPORTANTE ⚠️
Como fizemos a DUPLICAÇÃO de vários dados, ao tirar a média de Iterações por exemplo será fornecido um dado falso, pois antes tínhamos 5 linhas com esses valores, agora podemos ter 6,7,8 linhas o que interfere DIRETAMENTE no resultado

In [8]:
base.groupby("Tags")[["Curtidas"]].mean()

,Curtidas
Tags,
Datas comemorativas,"20,752.25"
Dicas de como usar,"6,833.40"
Influenciadores,"15,197.29"
Loja,"2,865.00"
Novos Produtos,"10,304.89"
Produtos,"6,269.82"
Promoções,"27,458.33"
Trends,"20,024.00"


In [9]:
# Ordenando por curtidas
base.groupby("Tags")[["Curtidas", "Comentários"]].mean().sort_values("Curtidas", ascending=False)

,Curtidas,Comentários
Tags,,
Promoções,"27,458.33",531.00
Datas comemorativas,"20,752.25",343.50
Trends,"20,024.00",352.25
Influenciadores,"15,197.29",161.71
Novos Produtos,"10,304.89",198.56
Dicas de como usar,"6,833.40",133.60
Produtos,"6,269.82",94.12
Loja,"2,865.00",17.67


### E o que está sem tag ?

In [10]:
# Filtrando os valores sem tag
base[base.Tags.isnull()]

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes
11,Foto,2021-10-12,17831,391,NaN,S,S,NaN,18222
19,Foto,2021-12-12,16086,268,NaN,S,S,NaN,16354
29,Foto,2022-01-19,8612,142,NaN,S,N,NaN,8754
38,Foto,2022-02-15,17687,213,NaN,S,N,NaN,17900
41,Foto,2022-02-22,12530,90,NaN,S,N,NaN,12620
43,Foto,2022-03-04,24399,266,NaN,S,S,NaN,24665
49,Foto,2022-03-22,9087,106,NaN,S,S,NaN,9193
50,Foto,2022-03-26,16551,186,NaN,S,N,NaN,16737


In [11]:
# Atribuindo o texto "Sem tag" para as colunas onde a tag é NaN
base.loc[base["Tags"].isnull(), ["Tags"]] = "Sem tag"
base.loc[base["Carrossel"].isnull(), ["Carrossel"]] = "N"
base

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes
0,Foto,2021-09-11,2858,16,Loja,N,N,N,2874
1,Foto,2021-09-11,2930,28,Loja,N,N,N,2958
1,Foto,2021-09-11,2930,28,Produtos,N,N,N,2958
2,Foto,2021-09-11,2807,9,Loja,N,N,N,2816
3,Vídeo,2021-09-12,5115,49,Produtos,N,N,N,5164
4,Foto,2021-09-13,4392,45,Produtos,S,N,N,4437
5,Foto,2021-09-17,5359,62,Novos Produtos,N,S,S,5421
6,Foto,2021-09-19,21597,852,Novos Produtos,S,S,N,22449
7,Foto,2021-09-25,6346,33,Produtos,N,S,N,6379
8,Foto,2021-09-27,6355,89,Produtos,S,N,S,6444


In [12]:
# Mostrando novamente a tabela de curtidas por TAG
base.groupby("Tags")[["Curtidas", 'Comentários']].mean().sort_values("Curtidas", ascending=False)

,Curtidas,Comentários
Tags,,
Promoções,"27,458.33",531.00
Datas comemorativas,"20,752.25",343.50
Trends,"20,024.00",352.25
Sem tag,"15,347.88",207.75
Influenciadores,"15,197.29",161.71
Novos Produtos,"10,304.89",198.56
Dicas de como usar,"6,833.40",133.60
Produtos,"6,269.82",94.12
Loja,"2,865.00",17.67


In [13]:
# Voltando com o Nan pois nao queremos que esse resultado apareça na nossa tabela final
base.loc[base["Tags"] == "Sem tag", ["Tags"]] = np.nan
base

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes
0,Foto,2021-09-11,2858,16,Loja,N,N,N,2874
1,Foto,2021-09-11,2930,28,Loja,N,N,N,2958
1,Foto,2021-09-11,2930,28,Produtos,N,N,N,2958
2,Foto,2021-09-11,2807,9,Loja,N,N,N,2816
3,Vídeo,2021-09-12,5115,49,Produtos,N,N,N,5164
4,Foto,2021-09-13,4392,45,Produtos,S,N,N,4437
5,Foto,2021-09-17,5359,62,Novos Produtos,N,S,S,5421
6,Foto,2021-09-19,21597,852,Novos Produtos,S,S,N,22449
7,Foto,2021-09-25,6346,33,Produtos,N,S,N,6379
8,Foto,2021-09-27,6355,89,Produtos,S,N,S,6444


### Analisando as tags com pessoas e campanhas:

In [17]:
# Análise de Pessoas e Tag
base_media = base.groupby(["Pessoas", "Tags"])[["Curtidas", "Comentários"]].mean()
base_media

Curtidas  Comentários
Pessoas Tags                                      
N       Loja                 2,865.00        17.67
        Novos Produtos       5,359.00        62.00
        Produtos             4,450.67        60.78
S       Datas comemorativas 20,752.25       343.50
        Dicas de como usar   6,833.40       133.60
        Influenciadores     15,197.29       161.71
        Novos Produtos      10,923.12       215.62
        Produtos             8,316.38       131.62
        Promoções           27,458.33       531.00
        Trends              20,024.00       352.25

In [19]:
# Ordenar por curtidas
base_media.sort_values("Curtidas", ascending=False)

Curtidas  Comentários
Pessoas Tags                                      
S       Promoções           27,458.33       531.00
        Datas comemorativas 20,752.25       343.50
        Trends              20,024.00       352.25
        Influenciadores     15,197.29       161.71
        Novos Produtos      10,923.12       215.62
        Produtos             8,316.38       131.62
        Dicas de como usar   6,833.40       133.60
N       Novos Produtos       5,359.00        62.00
        Produtos             4,450.67        60.78
        Loja                 2,865.00        17.67

In [20]:
# Fazendo para campanhas e tag
base_media = base.groupby(["Campanhas", "Tags"])[["Curtidas", "Comentários"]].mean()
base_media

Curtidas  Comentários
Campanhas Tags                                      
N         Dicas de como usar   6,833.40       133.60
          Influenciadores      6,402.00        72.00
          Loja                 2,865.00        17.67
          Novos Produtos       9,937.00       136.33
          Produtos             5,895.93        97.67
          Promoções           15,940.00       612.00
          Trends              12,894.00       249.00
S         Datas comemorativas 20,752.25       343.50
          Influenciadores     18,715.40       197.60
          Novos Produtos      11,040.67       323.00
          Produtos             9,074.00        67.50
          Promoções           33,217.50       490.50
          Trends              22,400.67       386.67

In [22]:
base_media.sort_values("Curtidas", ascending=False)

Curtidas  Comentários
Campanhas Tags                                      
S         Promoções           33,217.50       490.50
          Trends              22,400.67       386.67
          Datas comemorativas 20,752.25       343.50
          Influenciadores     18,715.40       197.60
N         Promoções           15,940.00       612.00
          Trends              12,894.00       249.00
S         Novos Produtos      11,040.67       323.00
N         Novos Produtos       9,937.00       136.33
S         Produtos             9,074.00        67.50
N         Dicas de como usar   6,833.40       133.60
          Influenciadores      6,402.00        72.00
          Produtos             5,895.93        97.67
          Loja                 2,865.00        17.67

### Conclusões gerais
- Ter o rosto de outras pessoas é fundamental para um bom engajamento na publicação
    - Em todas as tags quando tem o rosto, o resultado foi melhor
- Criar campanhas ajuda muito na divulgação da marca
- Promoções tiveram um desempenho absurdamente maior que qualquer outra tag
    - Porém é uma TAG que pode ter muito custo para a loja, o que deve ser analisado
- Usar conteúdos que estão em trend também ajudam na divulgação da marca, mesmo a trend sendo de outros nichos
- A melhor maneira de mostrar produtos é através de outras pessoas utilizando-os e se possível em campanhas de datas especiais
- Para novos produtos a inclus~]ao de pessoas é mais crítica ainda, sendo quase o dobro quando há um rosto junto ao produto
- Não podemos afirmar que a tag "LOJA" é rim até testarmos essa tag incluindo pessoas ou em uma campanha. Valo o teste para verificar
- Continuaremos a monitorar as postagens para encontrar novos padrões dado que ainda temos poucas informações da base